In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pickle
with open('outputs/non_constant_columns.pkl', 'rb') as f:
    non_constant_columns = pickle.load(f)

In [ ]:
target_df = pd.read_pickle('outputs/target_descriptors_calculated_n_processed.pkl')
target_df.head()

In [ ]:
target_df[target_df['Type'] == 'External Validation']

In [5]:
X_test = target_df[target_df['Type'] == 'External Validation'][non_constant_columns]
y_test = target_df[target_df['Type'] == 'External Validation']['HOMO_UPS']

In [ ]:
X_test.shape, y_test.shape

In [7]:
from joblib import dump, load
base_model = load('outputs/models/homo_dft.joblib')

In [8]:
from joblib import dump, load
homo_ups_predictor = load('outputs/models/homo_ups_predictor.joblib')

In [ ]:
homo_dft_predictions = base_model.predict(X_test)
homo_dft_predictions

In [ ]:
# the features names random forest trained on
homo_ups_predictor.feature_names_in_

In [ ]:
descriptors_used_for_training = list(homo_ups_predictor.feature_names_in_)[1:]
descriptors_used_for_training

In [ ]:
X_selected = X_test[descriptors_used_for_training]
X_selected.shape

In [13]:
X_selected.insert(0, 'DFT_pred', homo_dft_predictions)

In [ ]:
X_selected

In [ ]:
preds = homo_ups_predictor.predict(X_selected)
preds

In [ ]:
results_df = target_df[target_df['Type'] == 'External Validation']
results_df = results_df[['Molecule', 'HOMO_UPS']]
results_df['HOMO_UPS_prediction'] = preds
results_df['Error'] = np.abs(results_df['HOMO_UPS_prediction'] - results_df['HOMO_UPS'])
results_df

In [ ]:
# Mean Absolute Error
np.mean(results_df['Error'])